In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

# CCL name cleaning GDSC

In [100]:
gdsc_dr = pd.read_csv('data/GDSC/metrics.csv').rename(columns={'Cell_line':'CCL'})
gdsc_ge = pd.read_csv('data/GDSC/ge.csv', sep='\t').rename(columns={'Unnamed: 0':'CCL'}).transpose()


In [101]:
def clean(s):
    new = []
    for i in s:
        new.append(re.sub("[^A-Za-z0-9]", '', i).upper())
    return new

In [102]:
poz_r = list(gdsc_dr['CCL'].unique())
cell_r = list(gdsc_ge.index.to_series().reset_index()[0])

cell = clean(cell_r)
poz = clean(poz_r)

In [103]:
miss1 = []
for ccl in poz:
    if ccl not in cell:
        miss1.append(ccl)

miss1.remove('TTTHYROID')
print(len(miss1))
miss1

7


['786O', 'COLO320', 'EOL1', 'KNS81', 'NCIH510', 'U266B1', 'UO31']

In [104]:
poz2 = {k:re.sub("[^A-Za-z0-9]", '', k).upper() for k in poz_r if re.sub("[^A-Za-z0-9]", '', k).upper() not in miss1}

miss1_sub = ['7860', 'COLO320HSR','EOL1CELL', 'KNS81FD', 'NCIH510A' , 'U266', 'U031']

for i, ele in enumerate(miss1):
    poz2[ele] = miss1_sub[i]

In [105]:
gdsc_dr['CCL'] = gdsc_dr['CCL'].replace(poz2)
gdsc_dr = gdsc_dr[gdsc_dr['CCL']!='TTTHYROID']

In [106]:
gdsc_dr.to_csv('data/Processed/gdsc_poz_dr.csv', index =False)

In [82]:
gdsc_ge = gdsc_ge.reset_index()
gdsc_ge.columns = gdsc_ge.iloc[0]
gdsc_ge = gdsc_ge[1:]

In [83]:
cell_sub = {k:re.sub("[^A-Za-z0-9]", '', k).upper() for k in gdsc_ge['CCL']}
gdsc_ge['CCL'] = gdsc_ge['CCL'].replace(cell_sub)

In [84]:
drop = ['index']
for i in gdsc_ge['CCL']:
    if i not in gdsc_dr['CCL'].unique():
        drop.append(i)
gdsc_ge = gdsc_ge[~gdsc_ge['CCL'].isin(drop)].reset_index()

In [85]:
gdsc_ge.set_index('CCL').drop('index', axis=1).loc['KMH2']

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A3GALT2P,A4GALT,A4GNT,AAAS,AACS,...,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
CCL,,,,,,,,,,,,,,,,,,,,,
KMH2,5.57091,2.9072,3.82674,2.81857,3.24614,3.24614,3.16653,3.35596,5.29611,3.98259,...,7.44839,8.26429,9.63864,4.21265,4.64859,4.58405,4.62811,5.06164,4.7705,8.02462
KMH2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
gdsc_ge = gdsc_ge.drop_duplicates('CCL', keep='first')

In [99]:
gdsc_ge.drop('index', axis=1).to_csv('data/Processed/gdsc_cell_ge.csv', index=False)

# CCL name cleaning CCLE

In [88]:
ccle_ge = pd.read_csv('data/CCLE/ge.csv', sep='\t').rename(columns={'Unnamed: 0':'CCL'}).transpose()
ccle_dr = pd.read_csv('data/CCLE/metrics.csv').rename(columns={'Cell_line':'CCL'})

In [89]:
poz = list(ccle_dr['CCL'].unique())
cell = list(ccle_ge.index.to_series().str.upper().reset_index()[0])

In [90]:
poz = clean(poz)
cell = clean(cell)

In [91]:
miss2 = []
for ccl in poz:
    if ccl not in cell:
        miss2.append(ccl)
miss2

['BGC823',
 'GLC82',
 'KP1N',
 'KP1NL',
 'M059J',
 'MB157',
 'OVCAR3',
 'SF8657',
 'SNUC2B']

In [92]:
ccle_dr_2 = ccle_dr[~ccle_dr['CCL'].isin(['BGC823', 'GLC82', 'KP1N', 'KP1NL', 'M059J', 'SF8657', 'SNUC2B', 'MB157'])]
ccle_dr_2.to_csv('data/Processed/ccle_poz_dr.csv', index =False)

In [93]:
ccle_ge = ccle_ge.reset_index()
ccle_ge.columns = ccle_ge.iloc[0]
ccle_ge = ccle_ge[1:]

In [94]:
cell_sub = {k:re.sub("[^A-Za-z0-9]", '', k).upper() for k in ccle_ge['CCL'].unique()}
cell_sub['NIH:OVCAR-3'] = 'OVCAR3'
ccle_ge['CCL'] = ccle_ge['CCL'].replace(cell_sub)

In [95]:
drop = ['index']
for i in ccle_ge['CCL']:
    if i not in ccle_dr['CCL'].unique():
        drop.append(i)
ccle_ge = ccle_ge[~ccle_ge['CCL'].isin(drop)].reset_index()

In [21]:
ccle_ge.drop('index', axis=1).to_csv('data/Processed/ccle_cell_ge.csv', index =False)

# CCL name cleaning CTRP

In [22]:
ctrp_ge = pd.read_csv('data/CTRP/cell_ge.csv', sep='\t').rename(columns={'Unnamed: 0':'CCL'}).transpose()
ctrp_dr = pd.read_csv('data/CTRP/metrics.csv').rename(columns={'Cell_line':'CCL'})

In [23]:
import re

poz_r = list(ctrp_dr['CCL'].unique())
poz = clean(poz_r)

In [24]:
miss = []
for ccl in poz:
    if ccl not in cell:
        miss.append(ccl)
len(miss)

49

In [25]:
miss

['2004',
 'ASKA',
 'BT112',
 'BT131',
 'BT139',
 'BT145',
 'BT147',
 'BT159',
 'BT16',
 'BT164',
 'BT172',
 'BT179',
 'BT216',
 'BT224',
 'BT228',
 'BT231',
 'BT232',
 'BT239',
 'BT245',
 'BT248',
 'BT271',
 'BT286',
 'BT320',
 'BT328',
 'BT330',
 'BT333',
 'BT340',
 'BT359',
 'BT416',
 'BT422',
 'BT428',
 'BT440',
 'BT444',
 'BT482',
 'BT498',
 'BT504',
 'H292',
 'HS578BST',
 'HS888LU',
 'HSTS',
 'KRIJ',
 'PSTS',
 'SW982',
 'SYO1',
 'TM8716',
 'TTTHYROID',
 'UCH1',
 'YAMATO',
 'BT187']

We will drop all CCL's of the series BT and some others not found on cellminercdb

In [26]:
drop = [v for v in miss if v[:2]=='BT']

In [27]:
drop.extend(['2004', 'ASKA', 'HSTS', 'KRIJ', 'PSTS', 'SW982', 'SYO1', 'TM8716', 'UCH1', 'YAMATO'])

In [28]:
poz2 = {k:re.sub("[^A-Za-z0-9]", '', k) for k in poz_r if re.sub("[^A-Za-z0-9]", '', k).upper() not in miss}

miss1_sub = ['NCIH292', 'HS578T', 'HS888T', 'TT']

miss = [v for v in miss if v not in drop]
for i, ele in enumerate(miss):
    poz2[ele] = miss1_sub[i]

In [29]:
ctrp_dr = ctrp_dr[~ctrp_dr['CCL'].isin(drop)]
ctrp_dr['CCL'] = ctrp_dr['CCL'].replace(poz2)

In [30]:
ctrp_dr.to_csv('data/Processed/ctrp_poz_dr.csv', index =False)

In [31]:
ctrp_ge = ccle_ge
drop = ['index']
for i in ctrp_ge['CCL']:
    if i not in ctrp_dr['CCL'].unique():
        drop.append(i)
ctrp_ge = ctrp_ge[~ctrp_ge['CCL'].isin(drop)].reset_index()

In [32]:
ctrp_ge

,level_0,index,CCL,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,1,3,22RV1,4.75356,4.76688,7.44506,4.82488,5.71446,3.69636,4.50084,...,10.1581,11.8696,5.92065,5.41916,6.43494,4.58201,7.82893,6.26989,5.89303,8.19356
1,2,7,42MGBA,7.29013,5.5916,4.03057,4.1133,3.2771,3.9859,4.83985,...,10.1253,11.1474,5.0512,4.60405,5.91454,4.52143,8.25769,8.18264,6.07714,8.57538
2,3,8,5637,4.29347,4.51073,4.15791,4.1101,4.02171,4.16967,5.13869,...,9.32472,11.09,5.51127,5.11226,5.98138,5.20277,7.27636,8.61799,5.82305,7.84177
3,4,10,639V,6.62716,4.95192,4.09467,3.98008,4.31162,4.10599,4.64019,...,9.62425,11.1426,6.25403,5.55473,6.48561,6.701,8.50661,8.88766,6.40193,8.36718
4,5,12,697,6.99138,4.8361,4.02762,4.06956,4.35077,4.05418,4.69126,...,9.80042,11.6129,4.23948,5.21888,6.6696,4.83582,8.38729,6.82437,6.29979,8.26396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,491,1035,ZR751,6.69478,5.00265,3.75477,3.7694,5.61835,3.74015,4.62713,...,9.5565,11.6294,5.53224,5.13363,6.44712,7.03905,7.59483,5.96422,5.74333,7.75235
445,492,1036,ZR7530,6.14159,4.76631,3.82036,4.61768,4.34101,4.02952,4.87913,...,9.01199,10.6172,6.09086,5.60125,6.68985,6.92144,8.46681,7.64262,5.87172,7.9079
446,493,1067,OE21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447,494,1086,DOV13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
ctrp_ge.drop(['index', 'level_0'], axis=1).to_csv('data/Processed/ctrp_cell_ge.csv', index =False)